In [1]:
import os
import joblib # ou pickle

# --- BLOC DE CONFIGURATION DES CHEMINS (À METTRE AU DÉBUT) ---

# Le chemin vers le dossier 'models' est UN NIVEAU AU-DESSUS (../) du dossier 'notebooks'
MODELS_DIR = '../models'

# On s'assure que ce dossier existe. S'il n'existe pas, on le crée.
if not os.path.exists(MODELS_DIR):
    os.makedirs(MODELS_DIR)

# -----------------------------------------------------------------

In [2]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
import joblib
import os

print("Librairies importées avec succès.")

Librairies importées avec succès.


In [3]:
# Charger le dataset
df = pd.read_csv('data/dataset_compensation.csv')

# --- Prétraitement des données ---
# 1. Transformer les colonnes catégorielles en nombres
label_encoder_jour = LabelEncoder()
label_encoder_decision = LabelEncoder()

df['type_jour_encoded'] = label_encoder_jour.fit_transform(df['type_jour'])
df['decision_encoded'] = label_encoder_decision.fit_transform(df['decision'])

# 2. Afficher les mappings pour s'y retrouver
print("Mapping pour 'type_jour':", dict(zip(label_encoder_jour.classes_, label_encoder_jour.transform(label_encoder_jour.classes_))))
print("Mapping pour 'decision':", dict(zip(label_encoder_decision.classes_, label_encoder_decision.transform(label_encoder_decision.classes_))))

# 3. Définir les features (X) et la cible (y)
features = ['type_jour_encoded', 'duree_travaillee_heures', 'solde_repos_actuel_heures']
target = 'decision_encoded'

X = df[features]
y = df[target]

print("\nDonnées prêtes pour l'entraînement.")
df.head()

Mapping pour 'type_jour': {'FERIE': np.int64(0), 'REPOS': np.int64(1)}
Mapping pour 'decision': {'PAYE': np.int64(0), 'REPOS': np.int64(1)}

Données prêtes pour l'entraînement.


,type_jour,duree_travaillee_heures,solde_repos_actuel_heures,decision,type_jour_encoded,decision_encoded
0,FERIE,8,2,PAYE,0,0
1,REPOS,4,16,PAYE,1,0
2,FERIE,8,0,REPOS,0,1
3,REPOS,8,1,REPOS,1,1
4,FERIE,4,8,PAYE,0,0


In [4]:
# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)

# Créer et entraîner le modèle LightGBM (très efficace pour ce type de données)
# On utilise les noms de features pour que le modèle les mémorise
X_train.columns = features
lgbm = lgb.LGBMClassifier(objective='binary', random_state=42)
lgbm.fit(X_train, y_train)

# Évaluer la performance du modèle
y_pred = lgbm.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Précision du modèle sur l'ensemble de test : {accuracy:.2%}")
print("\nRapport de classification :")
print(classification_report(y_test, y_pred, target_names=label_encoder_decision.classes_))

[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Number of positive: 15, number of negative: 15
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 30, number of used features: 0
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirement

C:\Users\hjirt\anaconda1\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\hjirt\anaconda1\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\hjirt\anaconda1\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [5]:
# =================================================================
#                 CELLULE DE SAUVEGARDE (CORRIGÉE)
# =================================================================

# --- 1. Sauvegarde du modèle principal ---
# On utilise la variable "lgbm" qui a été créée dans la cellule d'entraînement.
model_path = os.path.join(MODELS_DIR, 'model_compensation.pkl')
joblib.dump(lgbm, model_path)
print(f"Modèle sauvegardé avec succès dans : {model_path}")

# --- 2. Sauvegarde des encodeurs ---
# On utilise aussi MODELS_DIR pour s'assurer que les encodeurs sont sauvegardés au bon endroit.
encoder_jour_path = os.path.join(MODELS_DIR, 'encoder_jour_type.pkl')
joblib.dump(label_encoder_jour, encoder_jour_path)
print(f"Encodeur 'jour_type' sauvegardé dans : {encoder_jour_path}")

encoder_decision_path = os.path.join(MODELS_DIR, 'encoder_decision_compensation.pkl')
joblib.dump(label_encoder_decision, encoder_decision_path)
print(f"Encodeur 'decision_compensation' sauvegardé dans : {encoder_decision_path}")

print("\nSauvegarde terminée.")

Modèle sauvegardé avec succès dans : ../models\model_compensation.pkl
Encodeur 'jour_type' sauvegardé dans : ../models\encoder_jour_type.pkl
Encodeur 'decision_compensation' sauvegardé dans : ../models\encoder_decision_compensation.pkl

Sauvegarde terminée.
